In [ ]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import LeaveOneOut, StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score, confusion_matrix, plot_roc_curve, roc_curve
from mlxtend.plotting import plot_confusion_matrix
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
from scipy.stats import ttest_ind
from mne.viz import circular_layout, plot_connectivity_circle
% matplotlib inline

import warnings
warnings.filterwarnings('ignore')

load data

labels: a file including 0 or 1 for each participant

features: a matrix in which each row is a participant, and each column is a feature. in this study, features are edges of a functional connectome, derived from the Schafer 7_networks_100_parcels parcellation

In [ ]:
labels = pd.read_csv('path_to_labels_file', sep='\n', header=None).squeeze()

features = pd.read_csv('path_to_features_file', index_col=0, header=0)

# scale features
scaler = StandardScaler()
features = pd.DataFrame(data = scaler.fit_transform(features), index=features.index, columns=features.columns)
features.shape

fit model to optimize parameters

In [ ]:
kfold=StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
log_lambda = np.linspace(3,-3,101)
C = 1/np.exp(log_lambda)
l1_ratios=np.linspace(0.7,1,7)
clf = LogisticRegressionCV(Cs=C,penalty='elasticnet',solver='saga', l1_ratios=l1_ratios,cv=kfold, scoring="roc_auc")
results = clf.fit(features, labels)
scores = results.scores_[True]

In [ ]:
best_l1_ratio = results.l1_ratio_[0]
print(f'best l1_ratio is: {best_l1_ratio}')
best_l1_loc = np.where(l1_ratios==best_l1_ratio)[0][0]
scores_2d = scores[:,:,best_l1_loc]

In [ ]:
mean_auc = np.mean(scores_2d, axis=0)
standard_deviations = np.std(scores_2d, axis=0)
standard_errors = standard_deviations / np.sqrt(10)
figure = plt.figure(figsize=(15, 5))
axes = figure.add_subplot(111)
axes.set_xlabel('ln(λ)')
axes.set_ylabel('AUC')
plt.errorbar(x=log_lambda, y=mean_auc, yerr=standard_errors)
plt.title('roc-auc scores for depending on regularization strength')

In [ ]:
best_log_lambda = -0.25 # determined by the plot
best_C = 1/np.exp(best_log_lambda)
best_l1_ratio=0.7
print(f'best log_lambda: {best_log_lambda}, best C: {best_C}')

In [ ]:
# define some functions for finding optimal threshold through ROC

def find_optimal_cutoff(target, predicted):
  # finds the optimal threshold from the ROC-curve
  fpr, tpr, threshold = roc_curve(target, predicted)
  i = np.arange(len(tpr)) 
  roc = pd.DataFrame({'tf' : pd.Series(tpr-(1-fpr), index=i), 'threshold' : pd.Series(threshold, index=i)})
  roc_t = roc.iloc[(roc.tf-0).abs().argsort()[:1]]

  return roc_t['threshold'].values[0]

def calc_precision(proba, Y_test, thresh, to_print=False):
  # calcultes the presicion of a model, given the prediction probabilities for each subject
  # set plot=True to plot the two populations' prediction probabilites
  proba = np.array(proba)
  predict_proba = proba>=thresh
  tn, fp, fn, tp = confusion_matrix(Y_test, predict_proba).ravel()
  accuracy = (tp+tn)/(tp+fp+tn+fn)
  sensitivity = tp / (tp+fn)
  specificity = tn / (tn+fp)
  if to_print:
    print(f'accuracy: {accuracy}\nsensitivity: {sensitivity}\nspecificity: {specificity}')
  return accuracy, sensitivity, specificity

def get_measures_by_kfold(features, labels, to_plot=False):
  kfold=StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
  clf = LogisticRegression(penalty='elasticnet', l1_ratio=best_l1_ratio, C=best_C, solver='saga', random_state=42)
  aucs = []
  accuracies = []
  sensitivities = []
  specificities = []
  if to_plot:
    fig, ax = plt.subplots()
  for fold, (train_index, test_index) in enumerate(kfold.split(features, labels)):
      X_train, X_test = features.iloc[train_index,:], features.iloc[test_index,:]
      Y_train, Y_test = labels[train_index], labels[test_index]
      clf.fit(X_train, Y_train)
      proba = clf.predict_proba(X_test)[:,1]
      aucs.append(roc_auc_score(Y_test, proba))
      optimal_thresh = find_optimal_cutoff(Y_test, proba)
      accuracy, sensitivity, specificity = calc_precision(proba, Y_test, optimal_thresh)
      accuracies.append(accuracy)
      sensitivities.append(sensitivity)
      specificities.append(specificity)
      if to_plot:
          plot_roc_curve(clf, X_test, Y_test,
                         name=f'ROC fold {fold+1}',
                         alpha=0.4, lw=3, ax=ax, linewidth=2)
          plt.plot([0,1],[0,1], color='black', linestyle='--')
  return np.mean(aucs), np.mean(accuracies), np.mean(sensitivities), np.mean(specificities)


In [ ]:
auc, accuracy, sensitivity, specificity = get_measures_by_kfold(features, labels, 1)
print(f'auc: {auc}\naccuracy: {accuracy}\nsensitivity: {sensitivity}\nspecificity: {specificity}')

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(features, labels, test_size=0.5, random_state=0, stratify=labels)

clf=LogisticRegression(penalty='elasticnet', l1_ratio=best_l1_ratio, C=best_C, solver='saga').fit(X_train, y_train)
auc_perm_scores = []
accuracy_perm_scores = []
sensitivity_perm_scores = []
specificity_perm_scores = []
test_pred_proba = clf.predict_proba(X_test)[:,1]

# first we calc the real values
optimal_threshold = find_optimal_cutoff(y_test, test_pred_proba)
accuracy, sensitivity, specificity = calc_precision(test_pred_proba, y_test, optimal_threshold)
auc = roc_auc_score(y_test, test_pred_proba)
auc_perm_scores.append(auc)
accuracy_perm_scores.append(accuracy)
sensitivity_perm_scores.append(sensitivity)
specificity_perm_scores.append(specificity)

# define a shuffling function

def shuffle_copy(to_shuffle):
  # returns shuffled DataFrame to utilize in permutation tests
  shuffled = to_shuffle.copy()
  shuffled = shuffled.values
  np.random.shuffle(shuffled)
  return shuffled

# now run the permutations

n_perm = 5000
for n in range(n_perm-1):
  shuffled_labels = shuffle_copy(y_test)
  optimal_threshold = find_optimal_cutoff(shuffled_labels, test_pred_proba)
  accuracy, sensitivity, specificity = calc_precision(test_pred_proba, shuffled_labels, optimal_threshold)
  auc = roc_auc_score(shuffled_labels, test_pred_proba)
  auc_perm_scores.append(auc)
  accuracy_perm_scores.append(accuracy)
  sensitivity_perm_scores.append(sensitivity)
  specificity_perm_scores.append(specificity)

permutation_df = pd.DataFrame(data=zip(auc_perm_scores,accuracy_perm_scores,sensitivity_perm_scores,specificity_perm_scores), columns=['AUC', 'Accuracy', 'Sensitivity', 'Specificity'])


In [ ]:
# calculate significance

def get_significance_perm(df,param):
  # calculate significance for permutation scores. assumes the true scores are the first in the data
  perm_scores = df[param]
  true_score = perm_scores[0]
  greater_eq_than = sum(perm_scores>=true_score)
  return round(greater_eq_than/len(perm_scores),4)

print('permutation test significance:\n')
for param in permutation_df.columns:
  print(f'{param} permutation scores: {get_significance_perm(permutation_df, param)}')

In [ ]:
#get beta values using mean of 1000 iterations

n_iters_coeff = 1000
coefs= np.zeros((n_iters_coeff,features.shape[1]))
for n in range(n_iters_coeff):
  print(n)
  clf = LogisticRegression(penalty='elasticnet', l1_ratio=best_l1_ratio, C=best_C, solver='saga')
  clf.fit(features, labels)
  coefs[n,:] = clf.coef_.flatten()
coef=np.mean(coefs, axis=0)


In [ ]:
# prepare data for plotting
dyads = []
for i in range(len(features.columns)):
  feat_data = features.columns[i].split('--X--')
  feat_data.append(coef[i])
  dyads.append(feat_data)
features_beta_df = pd.DataFrame(dyads, columns=['feature_1', 'feature_2', 'beta'])

In [ ]:
node_names = list(features_beta_df.feature_1) + list(features_beta_df.feature_2)
node_names = list(set(node_names))
con = np.abs(coef)

i_1 = []
i_2 = []
for f in range(len(features_beta_df)):
  feature = features_beta_df.iloc[f,:]
  i_1.append(node_names.index(feature['feature_1']))
  i_2.append(node_names.index(feature['feature_2']))
indices = (np.array(i_1), np.array(i_2))

lh_names = [name for name in node_names if 'LH' in name]
rh_names = [name for name in node_names if 'RH' in name]
lh_ordered = []
rh_ordered = []
net_names = ['Vis', 'SomMot', 'DorsAttn', 'SalVentAttn', 'Limbic', 'Cont', 'Default']
for network in net_names: 
  lh_net_names = [name for name in lh_names if network in name]
  lh_ordered = lh_ordered + lh_net_names

  rh_net_names = [name for name in rh_names if network in name]
  rh_ordered = rh_ordered + rh_net_names
lh_ordered.reverse()
ordered = lh_ordered + rh_ordered

layout = circular_layout(node_names=node_names, node_order=ordered, group_boundaries=[0, len(lh_ordered)])
colors_list = ['purple', 'blue', 'green', 'violet', 'wheat', 'orange', 'red']
#colors_list = [(120/255, 18/255, 133/255), (70/255, 130/255, 180/255), (0/255, 118/255, 14/255), (196/255, 57/255, 248/255), (220/255, 248/255, 162/255), (230/255, 146/255, 32/255), (204/255, 60/255, 78/255)]

color_dict = dict(zip(net_names, colors_list))
node_colors = []
for name in node_names:
  net = name.split('_')[2]
  node_colors.append(color_dict[net])
